# Plot

In [ ]:
from pathlib import Path
from PIL import Image

In [ ]:
paths = sorted(Path("/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg").rglob("*.png"))
print(len(paths))

In [ ]:
path = str(paths[0])
display(Image.open(path).resize((256, 256)))
display(Image.open(path.replace("seg", "rgb")).resize((256, 256)))
display(Image.open(path.replace("seg", "ir")).resize((256, 256)))

In [ ]:
path = str(paths[1])
display(Image.open(path).resize((256, 256)))
display(Image.open(path.replace("seg", "rgb")).resize((256, 256)))
display(Image.open(path.replace("seg", "ir")).resize((256, 256)))

In [ ]:
path = str(paths[2])
display(Image.open(path).resize((256, 256)))
display(Image.open(path.replace("seg", "rgb")).resize((256, 256)))
display(Image.open(path.replace("seg", "ir")).resize((256, 256)))

In [ ]:
path = str(paths[3])
display(Image.open(path).resize((256, 256)))
display(Image.open(path.replace("seg", "rgb")).resize((256, 256)))
display(Image.open(path.replace("seg", "ir")).resize((256, 256)))

In [ ]:
path = str(paths[4])
display(Image.open(path).resize((256, 256)))
display(Image.open(path.replace("seg", "rgb")).resize((256, 256)))
display(Image.open(path.replace("seg", "ir")).resize((256, 256)))

# SAM

In [ ]:
import os
from typing import Any, Dict, List

import torch

import numpy as np
from PIL import Image
from tqdm import tqdm

from sam2.build_sam import build_sam2
from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

In [ ]:
device = torch.device("cuda")

sam2_checkpoint = "/data/hanchong/DiffV2IR/sam2/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

#sam2_checkpoint = "/data/hanchong/DiffV2IR/sam2/sam2.1_hiera_tiny.pt"
#model_cfg = "configs/sam2.1/sam2.1_hiera_t.yaml"

sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device, apply_postprocessing=False)

In [ ]:
def write_masks_to_folder(masks: List[Dict[str, Any]], path: str) -> None:
    os.makedirs(path, exist_ok=True)
    header = "id,area,bbox_x0,bbox_y0,bbox_w,bbox_h,point_input_x,point_input_y,predicted_iou,stability_score,crop_box_x0,crop_box_y0,crop_box_w,crop_box_h"  # noqa
    metadata = [header]
    for i, mask_data in enumerate(masks):
        mask = mask_data["segmentation"]
        filename = f"{i}.png"
        Image.fromarray(mask).save(os.path.join(path, filename))
        mask_metadata = [
            str(i),
            str(mask_data["area"]),
            *[str(x) for x in mask_data["bbox"]],
            *[str(x) for x in mask_data["point_coords"][0]],
            str(mask_data["predicted_iou"]),
            str(mask_data["stability_score"]),
            *[str(x) for x in mask_data["crop_box"]],
        ]
        row = ",".join(mask_metadata)
        metadata.append(row)
    metadata_path = os.path.join(path, "metadata.csv")
    with open(metadata_path, "w") as f:
        f.write("\n".join(metadata))

    return

In [ ]:
paths = [
    #"/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg/AVIID1_0.png",
    #"/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg/AVIID1_1.png",
    #"/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg/AVIID1_10.png",
    #"/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg/AVIID1_100.png",
    #"/data/hanchong/DiffV2IR/IR-500K/IR-500k/visible-infrared_pairs/seg/AVIID1_1000.png",
    "/data/hanchong/images-splitted-3/In-distribution/val/C00/10567.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C02/171181.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C04/6146.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C05/9160.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C07/251504.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C08/32724.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C09/76585.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C10/22241.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C11/131560.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C12/87263.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C13/7781.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C14/388.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C15/62610.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C17/86659.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C18/3528.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C19/74553.jpg",
    "/data/hanchong/images-splitted-3/In-distribution/val/C20/479692.jpg",

]

In [ ]:
mask_generator = SAM2AutomaticMaskGenerator(sam2)

for path in tqdm(paths):
    image = np.array(Image.open(path).convert("RGB"))
    masks = mask_generator.generate(image)
    write_masks_to_folder(masks, f"/data/hanchong/DiffV2IR/test/test-1/auto-masks/{os.path.basename(path).split('.')[0]}")

In [ ]:
mask_generator_2 = SAM2AutomaticMaskGenerator(
    model=sam2,
    points_per_side=64,
    points_per_batch=128,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.92,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.7,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=25.0,
    use_m2m=True,
)  # Too slow

for path in tqdm(paths):
    image = np.array(Image.open(path).convert("RGB"))
    masks2 = mask_generator_2.generate(image)
    write_masks_to_folder(masks2, f"/data/hanchong/DiffV2IR/test/test-2.1/auto-masks/{os.path.basename(path).split('.')[0]}")

# 23m 20s, large

# Data Prep

In [2]:
from pathlib import Path
import pandas as pd

In [3]:
input_folder = Path("/data/hanchong/maritime-vessel-dataset-infrared/In-distribution/val")
input_paths = sorted(input_folder.rglob("*.jpg"))
print(len(input_paths))

output_folder = Path("/data/hanchong/maritime-vessel-dataset-infrared/In-distribution_Auto-Masks/val")
output_paths = [output_folder / path.parent.name / path.stem for path in input_paths]


1700


In [4]:
df = pd.DataFrame({"input": input_paths, "output": output_paths})
df.head()

,input,output
0,/data/hanchong/maritime-vessel-dataset-infrare...,/data/hanchong/maritime-vessel-dataset-infrare...
1,/data/hanchong/maritime-vessel-dataset-infrare...,/data/hanchong/maritime-vessel-dataset-infrare...
2,/data/hanchong/maritime-vessel-dataset-infrare...,/data/hanchong/maritime-vessel-dataset-infrare...
3,/data/hanchong/maritime-vessel-dataset-infrare...,/data/hanchong/maritime-vessel-dataset-infrare...
4,/data/hanchong/maritime-vessel-dataset-infrare...,/data/hanchong/maritime-vessel-dataset-infrare...


In [5]:
df.iloc[0].input

PosixPath('/data/hanchong/maritime-vessel-dataset-infrared/In-distribution/val/C00/1043276.jpg')

In [6]:
df.iloc[0].output

PosixPath('/data/hanchong/maritime-vessel-dataset-infrared/In-distribution_Auto-Masks/val/C00/1043276')

In [7]:
for i in range(0, len(df), 200000):
    _df = df.iloc[i : i + 200000]
    _df.to_csv(f"/data/hanchong/maritime-vessel-dataset-infrared/In-distribution_Auto-Masks/val_batch_{i//200000}.csv", index=False)